## LSTM Network Training

In [ ]:
import os
import warnings
import pandas as pd

from src.StockPredictionConfig import StockPredictionConfig
from src.StockDataProcessor import StockDataProcessor
from src.LongShortTermMemory import LSTMModel

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or '3' to suppress all messages

# Suppress other warnings
warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")

from datetime import timedelta

In [ ]:
FOLDER_PREFIX = "data/min/"
STOCK_START_DATE = pd.to_datetime("2017-06-07 15:59:00")
# STOCK_START_DATE = pd.to_datetime("2024-05-24 09:30:00")
STOCK_VALIDATION_DATE = pd.to_datetime("2024-06-09 09:30:00")
STOCK_END_DATE = pd.to_datetime("2024-06-12 15:59:00")
TIME_STEPS = 60

In [ ]:
# FOLDER_PREFIX = "data/day/"
# STOCK_START_DATE = pd.to_datetime("2017-06-07")
# STOCK_VALIDATION_DATE = pd.to_datetime("2022-06-07")
# STOCK_END_DATE = pd.to_datetime("2024-06-07")
# TIME_STEPS = 1

In [ ]:
# import secrets
# TODAY_RUN = datetime.today().strftime(DATE_FORMAT)
# TOKEN = STOCK_TICKER + '_' + TODAY_RUN + '_' + secrets.token_hex(16)

In [ ]:
DATE_FORMAT = "%Y-%m-%d"
STOCK_TICKER = "GOOG"
EPOCHS = 100
BATCH_SIZE = 10
TOKEN = "GOOG"
RUN_FOLDER = f"{FOLDER_PREFIX}{TOKEN}/"
WORK_DIR = os.path.join(os.getcwd(), RUN_FOLDER)
CSV_FILE = f"{WORK_DIR}data.csv"

print('Ticker: ' + STOCK_TICKER)
print('Start Date: ' + STOCK_START_DATE.strftime(DATE_FORMAT))
print('Validation Date: ' + STOCK_VALIDATION_DATE.strftime(DATE_FORMAT))
print('Test Run Folder: ' + RUN_FOLDER)

PROJECT_FOLDER = os.path.join(os.getcwd(), RUN_FOLDER)
if not os.path.exists(PROJECT_FOLDER):
    os.makedirs(PROJECT_FOLDER)

In [ ]:
stock_config = StockPredictionConfig(STOCK_TICKER, 
                                   STOCK_START_DATE,
                                   STOCK_END_DATE,
                                   STOCK_VALIDATION_DATE, 
                                   PROJECT_FOLDER, 
                                   EPOCHS,
                                   TIME_STEPS,
                                   TOKEN,
                                   BATCH_SIZE)


In [ ]:
(x_train, y_train), (x_test, y_test), (training_data, test_data) = StockDataProcessor.load_csv_transform_to_numpy(TIME_STEPS, CSV_FILE, STOCK_VALIDATION_DATE)

In [ ]:
# Execute Deep Learning model
lstm = LSTMModel(stock_config.project_folder)
lstm.Train(stock_config, x_train, y_train, x_test, y_test, training_data, test_data)

## Prediction

In [ ]:
data = pd.read_csv(CSV_FILE)

latest_close_price = data['Close'].iloc[-1]
latest_date = data['Datetime'].iloc[-1]

print(f"{latest_close_price}: latest_close_price")
print(f"{latest_date}: latest_date")

start_date = pd.to_datetime(latest_date) + timedelta(1)
end_date = pd.to_datetime(start_date) + timedelta(1)
# end_date = pd.to_datetime(start_date) + timedelta(5)

print(f"{start_date}: start_date")
print(f"{end_date}: end_date")

In [ ]:
lstm.Infer(start_date, end_date, latest_close_price, WORK_DIR, TIME_STEPS, STOCK_TICKER, stock_config.CURRENCY)